# Automated Question-Answer Generation from Pharmaceutical Data

This notebook systematically generates question-answer pairs from pharmaceutical data sources including:
1. PubMed target passages
2. DrugBank tables
3. Related pharmaceutical data

Each QA pair will include:
- Question
- Answer
- Source text/passage
- Related table

In [1]:
# Import required libraries
import os
import pandas as pd
import json
from pathlib import Path
from openai import OpenAI
import csv
from tqdm import tqdm
from collections import defaultdict

In [2]:
# Configuration and paths
PUBMED_TARGETS_DIR = 'inputs/pubmed-targets'
DRUGBANK_TABLES_DIR = 'inputs/drugbank-tables'
MAPPING_FILE = 'inputs/pubmed-drugbank-tables.gt'
OUTPUT_FILE = 'table_output.gt'

# Initialize your LLM API key if needed
# os.environ["OPENAI_API_KEY"]
client = OpenAI()


In [3]:
def load_passage_table_mapping():
    """Load the mapping between passages and their relevant tables"""
    mapping = defaultdict(list)
    with open(MAPPING_FILE, 'r') as f:
        for line in f:
            passage_id, table_name = line.strip().split(',')
            mapping[passage_id].append(table_name)
    # print("mapping", mapping)
    return mapping

def load_target_passages():
    """Load all target passages from the pubmed-targets directory"""
    passages = {}
    target_files = Path(PUBMED_TARGETS_DIR).glob('Target-*')
    
    for file_path in target_files:
        target_id = file_path.name
        with open(file_path, 'r') as f:
            passages[target_id] = f.read()
    # print("passages", passages)
    return passages

def load_drugbank_tables():
    """Load all relevant DrugBank tables"""
    tables = {}
    csv_files = Path(DRUGBANK_TABLES_DIR).glob('*.csv')
    
    for file_path in csv_files:
        table_name = file_path.stem
        tables[table_name] = pd.read_csv(file_path)
    
    # print("tables", tables)
    return tables

In [4]:
def get_relevant_table_content(tables, table_names, max_rows=5):
    """Extract relevant content from tables for context"""
    print("Debug - Available tables:", tables.keys())
    print("Debug - Looking for table_names:", table_names)

    table_content = {}
    for table_name in table_names:
        # Remove .csv extension if present
        base_table_name = table_name.replace('.csv', '')
        
        if base_table_name in tables:
            df = tables[base_table_name]
            table_content[base_table_name] = {
                'columns': list(df.columns),
                'sample': df.head(max_rows).to_dict('records')
            }
        else:
            print(f"Debug - Table '{base_table_name}' not found in available tables")
    return table_content

def generate_questions_for_passage(passage_id, passage_text, tables, relevant_table_names, model="gpt-4o"):
    """Generate questions for a given passage and its relevant tables using LLM"""

    # print("tables", tables)
    # print("relevant_table_names", relevant_table_names)
    
    # Limit passage length if too long (e.g., first 1000 characters)
    passage_text = passage_text[:1000] + "..." if len(passage_text) > 1000 else passage_text
    
    # Limit to maximum 3 relevant tables
    relevant_table_names = relevant_table_names[:3]
    
    # Get relevant table content
    table_content = get_relevant_table_content(tables, relevant_table_names)

    
    prompt = f"""
    Given the following passage and related tables, generate 3-5 meaningful question-answer pairs.
    IMPORTANT: Each question MUST be answerable using information from ONLY the tables.
    Only generate questions that require information from the tables.
    
    You are a pharmaceutical educator creating realistic questions that a medical student, pharmacist, or healthcare professional might ask. Focus on clinically relevant information such as:
    - Drug mechanisms, indications, and therapeutic uses
    - Dosing considerations and administration routes
    - Side effects and contraindications
    - Drug interactions and pharmacological properties
    
    AVOID generating questions about:
    - Database-specific information (primary keys, IDs, URLs)
    - Technical metadata that wouldn't be useful in clinical practice
    - Questions that simply ask to list information without clinical context
    
    Examples of BAD questions to AVOID:
    - "What drug is linked to the FDA label with the URL ending in '1265922800,' and what is its state?"
    - "What is the primary key for the drug Denileukin diftitox, and what is its marketed mixture name?"
    
    Related Tables:
    {json.dumps(table_content, indent=2)}
    
    Generate questions in the following format:
    1. question: [specific question about drug/treatment that would be asked in clinical practice]
       answer: [detailed answer combining information from the tables]
       text: None
       table: [table name if information from table was used (e.g. drugbank-drug)]
    
    Ensure every question is clinically relevant and would be asked by real healthcare professionals.
    """
    
    # Call your LLM here with the prompt
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical and pharmaceutical expert tasked with generating detailed question-answer pairs about drugs and treatments."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    print(response.choices[0].message.content)
    
    # Parse the response into structured QA pairs
    qa_pairs = parse_llm_response(response.choices[0].message.content, passage_id)
    return qa_pairs

def parse_llm_response(response_text, passage_id):
    """Parse the LLM response into structured QA pairs"""
    qa_pairs = []
    
    # Split the response into individual QA entries
    entries = response_text.strip().split('\n\n')
    
    for entry in entries:
        if not entry.strip():
            continue
            
        lines = entry.strip().split('\n')
        current_qa = {
            'question': '',
            'answer': '',
            'text': passage_id,
            'table': 'None'  # Default value
        }
        
        for line in lines:
            line = line.strip()
            # Skip empty lines and numbering
            if not line or line.replace('.', '').strip().isdigit():
                continue
                
            # Parse each field using more robust splitting
            if 'question:' in line:
                current_qa['question'] = line.split('question:', 1)[1].strip()
            elif 'answer:' in line:
                current_qa['answer'] = line.split('answer:', 1)[1].strip()
            elif 'text:' in line:
                current_qa['text'] = line.split('text:', 1)[1].strip()
            elif 'table:' in line:
                table_value = line.split('table:', 1)[1].strip()
                # Handle NA, N/A, None cases
                current_qa['table'] = 'None' if table_value.upper() in ['NA', 'N/A', 'NONE'] else table_value
        
        # Only add complete QA pairs that have both question and answer
        if current_qa['question'] and current_qa['answer']:
            qa_pairs.append(current_qa.copy())  # Use copy to avoid reference issues
    
    return qa_pairs

In [10]:
def main():
    # Load mappings and data
    print("Loading passage-table mappings...")
    passage_table_mapping = load_passage_table_mapping()
    
    print("Loading target passages...")
    passages = load_target_passages()
    
    print("Loading DrugBank tables...")
    tables = load_drugbank_tables()
    
    # Initialize output list
    qa_pairs = []
    
    # Process each passage with its relevant tables
    for passage_id, passage_text in tqdm(list(passages.items())[:200]):
        if passage_id in passage_table_mapping:
            relevant_tables = passage_table_mapping[passage_id]
            # print(passage_id, passage_text, tables, relevant_tables)
            
            # Generate QA pairs using the passage and its relevant tables
            new_qa_pairs = generate_questions_for_passage(
                passage_id,
                passage_text,
                tables,
                relevant_tables
            )
            # new_qa_pairs = []
            print(new_qa_pairs)
            qa_pairs.extend(new_qa_pairs)
        else:
            print(f"No table mapping found for passage {passage_id}")
    
    # Save results
    with open(OUTPUT_FILE, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for qa_pair in qa_pairs:
            writer.writerow([
                qa_pair['question'],
                qa_pair['answer'],
                qa_pair['text'],
                qa_pair['table']
            ])
        
    print(f"Generated {len(qa_pairs)} question-answer pairs")

In [11]:
if __name__ == "__main__":
    main()

Loading passage-table mappings...
Loading target passages...
Loading DrugBank tables...


/tmp/ipykernel_35724/2028544267.py:30: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tables[table_name] = pd.read_csv(file_path)
  0%|          | 0/200 [00:00<?, ?it/s]

Debug - Available tables: dict_keys(['drugbank-drug_trans_links', 'drugbank-enzymes_polypeptides_ext_id', 'drugbank-drug_manufacturers', 'drugbank-drug_sequences', 'drugbank-drug_external_links', 'drugbank-drug_international_brands', 'drugbank-drug_mixtures', 'drugbank-drug_packagers', 'drugbank-drug_reactions_enzymes', 'drugbank-drug_enzymes_articles', 'drugbank-targets', 'drugbank-drug_carriers_textbooks', 'drugbank-drug_trans_textbooks', 'drugbank-carriers_polypeptides_ext_id', 'drugbank-drugs_attachments', 'drugbank-drug_carriers_articles', 'drugbank-enzymes_polypeptides', 'drugbank-drug_trans_articles', 'drugbank-drug_experimental_properties', 'drugbank-transporters_polypeptides_ext_id', 'drugbank-drug_dosages', 'drugbank-drug_syn', 'drugbank-drug_targ_textbooks', 'drugbank-drug_enzymes_attachments', 'drugbank-drug_carriers_links', 'drugbank-drug_salts', 'drugbank-drugs_articles', 'drugbank-targets_actions', 'drugbank-drug_patents', 'drugbank-drug_affected_organisms', 'drugbank-qu

  0%|          | 1/200 [00:07<23:15,  7.01s/it]

1. question: What is the mechanism of action for Cetuximab and which receptor does it target?
   answer: Cetuximab is an epidermal growth factor receptor (EGFR) binding fragment antigen-binding (FAB), targeting the EGFR in humans. Its mechanism of action involves binding to the N-terminal portion of human EGFR.
   text: None
   table: drugbank-targets

2. question: What are the components of Etanercept and how is it produced?
   answer: Etanercept is a dimeric fusion protein consisting of the extracellular ligand-binding portion of the human 75 kilodalton (p75) tumor necrosis factor receptor (TNFR) linked to the Fc portion of human IgG1. It is produced by recombinant DNA technology in a Chinese hamster ovary (CHO) mammalian cell expression system.
   text: None
   table: drugbank-drug

3. question: Which drug is associated with the FDA label titled "FDA Approved Drug Products: Lupron Depot (leuprolide acetate) depot suspension"?
   answer: The drug associated with the FDA label titled 

...

1. question: Which drugs are associated with the transporter P-glycoprotein 1, and what is their organism of action?
   answer: The drugs associated with the transporter P-glycoprotein 1 are linked to the parent keys DB00027 and DB00091. Both of these drugs have their action in humans.
   text: None
   table: drugbank-transporters

2. question: What is the description and state of the drug Cetuximab?
   answer: Cetuximab is an epidermal growth factor receptor binding FAB. It is composed of the Fv regions of the 225 murine EGFr monoclonal antibody specific for the N-terminal portion of human EGFr with human IgG1 heavy and kappa light chain constant regions. Cetuximab is marketed under the brand Erbitux® by Eli Lilly and Company. It is in a liquid state.
   text: None
   table: drugbank-drug

3. question: What is the mechanism of action for Dornase alfa, and what clinical condition does it help manage?
   answer: Dornase alfa is a biosynthetic form of the human deoxyribonuclease I enzyme

 99%|█████████▉| 198/200 [16:19<00:05,  2.95s/it]

1. question: What is the primary therapeutic mechanism of action for Cetuximab?
   answer: Cetuximab acts by binding to the epidermal growth factor receptor (EGFR) on human cells. This binding inhibits the receptor's function, which is involved in the proliferation of cancer cells.
   text: None
   table: drugbank-targets

2. question: Which drug is indicated for reducing sputum viscosity in cystic fibrosis patients, and what is its mechanism?
   answer: Dornase alfa is indicated for reducing sputum viscosity in patients with cystic fibrosis. It is a biosynthetic form of human deoxyribonuclease I (DNase I) enzyme that cleaves extracellular DNA, thereby reducing sputum viscosity and viscoelasticity.
   text: None
   table: drugbank-drug

3. question: What is the common production method used for the drug Denileukin diftitox?
   answer: Denileukin diftitox is produced using recombinant DNA technology in an E. coli expression system. It is a cytotoxic protein composed of sequences from di

100%|██████████| 200/200 [16:26<00:00,  4.93s/it]

1. question: What is the mechanism of action for Cetuximab in targeting cancer cells?
   answer: Cetuximab works by binding to the epidermal growth factor receptor (EGFR) on cancer cells. This binding inhibits the receptor's action, which is crucial for cancer cell proliferation and survival. By blocking EGFR, Cetuximab interferes with the growth and spread of cancer cells.
   text: None
   table: drugbank-targets

2. question: What type of drug is Dornase alfa, and what is its primary clinical use?
   answer: Dornase alfa is a biotech drug that acts as a biosynthetic form of human deoxyribonuclease I (DNase I) enzyme. It is primarily used to reduce sputum viscosity in patients with cystic fibrosis by enzymatically breaking down extracellular DNA present in the sputum.
   text: None
   table: drugbank-drug

3. question: Which drug listed is known to target prothrombin and what is its primary application?
   answer: Lepirudin targets prothrombin, which is a key protein in the coagulatio